In [1]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path =r"C:\Users\NT550-052\OneDrive\바탕 화면\ASAC\프로젝트\ML\web_crawling\CGW\finaldata\finaldata_gw.csv"
data = pd.read_csv(path,sep="\t")
data

,appid,24_Hour_Peak,All_time_peak,positive,negative,average_forever,average_2weeks,median_forever,median_2weeks,price,...,metacritic_tf,windows,mac,linux,platforms_num,recommendations,achievements,new_date,days_after_release,Review
0,294100,20930,60742,157344,3038,12492,1401,5368,921,34.99,...,1,1.0,1.0,1.0,3.0,134967.0,0.0,2018-10-17,1675,Overwhelmingly Positive
1,960090,13428,53891,246979,6441,4018,277,1532,62,13.99,...,0,1.0,1.0,0.0,2.0,222889.0,146.0,2018-12-18,1613,Overwhelmingly Positive
2,504230,1234,4148,71428,1686,1316,310,370,582,19.99,...,1,1.0,1.0,1.0,3.0,66397.0,32.0,2018-01-25,1940,Overwhelmingly Positive
3,588650,4553,14880,128085,3722,1841,183,868,213,14.99,...,1,1.0,1.0,1.0,3.0,113184.0,121.0,2018-08-06,1747,Overwhelmingly Positive
4,264710,4205,50876,221972,7800,1943,313,941,277,29.99,...,1,1.0,1.0,0.0,2.0,208334.0,17.0,2018-01-24,1941,Overwhelmingly Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46247,1995280,0,0,0,0,0,0,0,0,14.99,...,0,1.0,0.0,0.0,1.0,0.0,0.0,2022-12-02,168,Negative
46248,2098860,0,0,0,11,0,0,0,0,9.99,...,0,1.0,0.0,0.0,1.0,0.0,0.0,2022-08-09,283,Negative
46249,2154770,814,1176,0,0,0,0,0,0,0.00,...,0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-01,169,Very Negative
46250,1915410,0,1,0,16,0,0,0,0,2.99,...,0,1.0,0.0,0.0,1.0,0.0,0.0,2022-04-26,388,Negative


In [19]:
tag_path = r"C:\Users\NT550-052\OneDrive\바탕 화면\ASAC\프로젝트\ML\web_crawling\merge_dataset\tag_merge.csv"
tag_df = pd.read_csv(tag_path)
data = pd.merge(data,tag_df,how="inner",left_on="appid",right_on="appid")

In [20]:
data.drop(columns=["genre","appid"],inplace=True)

In [21]:
data

,24_Hour_Peak,All_time_peak,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,num_lang,required_age,...,360 Video,Linear,Grid-Based Movement,FMV,Lemmings,Tile-Matching,Indie,Arcade,Atmospheric,Cats
0,20930,60742,12492,1401,5368,921,34.99,34.99,35,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1223.0,0.0,0.0,0.0
1,13428,53891,4018,277,1532,62,13.99,13.99,24,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,161.0,0.0,0.0,0.0
2,1234,4148,1316,310,370,582,19.99,19.99,14,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,887.0,0.0,320.0,0.0
3,4553,14880,1841,183,868,213,14.99,24.99,21,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1003.0,0.0,3360.0,0.0
4,4205,50876,1943,313,941,277,29.99,29.99,38,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1420.0,0.0,2618.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46247,0,0,0,0,0,0,14.99,14.99,4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46248,0,0,0,0,0,0,9.99,9.99,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,322.0,0.0
46249,814,1176,0,0,0,0,0.00,0.00,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46250,0,1,0,0,0,0,2.99,2.99,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0


In [23]:
X = data.drop(columns="Review")
y = data["Review"]

In [29]:
from sklearn.model_selection import train_test_split 



X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size = 0.2,
    random_state = 1234, # 옵션, 재현성을 위함 
    stratify = y # 옵션 : 비율 나눌 때 비율 유지할지 말지 
)

In [25]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=0,sampling_strategy='all').fit_resample(X_train, y_train)

In [28]:
X_train,y_train = X_resampled,y_resampled

In [32]:
# str-kfold
from sklearn.model_selection import StratifiedKFold

str_kfold = StratifiedKFold(n_splits=5, random_state=1234, shuffle=True)

In [33]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,recall_score,f1_score

In [34]:
# lightgbm base model
from lightgbm import LGBMClassifier
from sklearn.metrics import make_scorer

# scorers = {
#             'f1_score': make_scorer(f1_score, average='micro'),
#             'recall_score': make_scorer(recall_score, average='micro'),
#             'accuracy_score': make_scorer(accuracy_score)
#           }

lgbm = LGBMClassifier(random_state=1234,n_jobs=-1)
scores = cross_val_score( lgbm, 
                         X_train, y_train,
                         cv = str_kfold, # --> 고정적으로 CV할 때 데이터셋 고정..
                         scoring = "accuracy")
for iter_count, acc in enumerate( scores):
    print("lgbm {0}번째 검증 ACC:{1:.4f}".format(iter_count+1,acc ))
print("lgbm 모델 Accuracy 평균:", scores.mean())
print("lgbm 모델 Accuracy std:", scores.std())


lgbm 1번째 검증 ACC:0.7197
lgbm 2번째 검증 ACC:0.7212
lgbm 3번째 검증 ACC:0.7244
lgbm 4번째 검증 ACC:0.7282
lgbm 5번째 검증 ACC:0.7289
lgbm 모델 Accuracy 평균: 0.7244769874476986
lgbm 모델 Accuracy std: 0.003673719177778356


In [35]:
lgbm_model = lgbm.fit(X_train,y_train)

In [36]:
lgbm_ypred = lgbm_model.predict(X_val)
lgbm_acc = accuracy_score(y_val,lgbm_ypred)
print("LGBM acc:",lgbm_acc)

LGBM acc: 0.7277291452111225


In [37]:
from sklearn.metrics import classification_report
print(classification_report(y_val,lgbm_ypred))

              precision    recall  f1-score   support

Indifference       0.69      0.81      0.74      3884
       Mixed       0.73      0.57      0.64      3884
    Negative       0.81      0.88      0.84      3884
    Positive       0.68      0.65      0.66      3884

    accuracy                           0.73     15536
   macro avg       0.73      0.73      0.72     15536
weighted avg       0.73      0.73      0.72     15536



In [38]:
pd.DataFrame([X_train.columns, lgbm_model.feature_importances_]).T.sort_values(by=1,ascending=False)

,0,1
19,days_after_release,538
468,Indie,448
7,initialprice,441
18,achievements,330
6,price,321
...,...,...
32,Musou,0
82,Looter Shooter,0
373,Reboot,0
120,Werewolves,0
